In [1]:
%load_ext autoreload
%autoreload 2



import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
SPEECH_DATA_PATH = "./speech/data/"

URBAN_NOISE_DATA_PATH = "./urban/data/"

COMBINED_DATA_PATH = "./combined/data/"

SPEECH_NPY_PATH = "./40bins_allfeat/npy/"
_
URBAN_NOISE_NPY_PATH = "./40bins_allfeat/npy/"

COMBINED_NPY_PATH =  "./combined/40bins_allfeat/npy/"

COMBINED_MODEL_SAVE_PATH = "./models/40bins__allfeat"


In [3]:
# speech test and val file paths
testFile = "./data/testing_list.txt"
valFile = "./data/validation_list.txt"


### Feature extraction and saving

In [4]:
##function to convert wav2mfcc
# samp rate of the speech wav files is 16000
# samp rate of the urban noise wav files is 8000
# use the samprate argument to change that.... will downsample speech wav files to 8000 in the future to make it easier
def wav2mfcc(file_path, max_len=11,samprate = 8000):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    ##mfcc = librosa.feature.mfcc(wave, sr=16000)
    mfcc = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=40)
    melspectrogram =librosa.feature.melspectrogram(y=wave, sr=sr, n_mels=40,fmax=10000)
    chroma_stft=librosa.feature.chroma_stft(y=wave, sr=sr,n_chroma=40)
    chroma_cq =librosa.feature.chroma_cqt(y=wave, sr=sr,n_chroma=40)
    chroma_cens =librosa.feature.chroma_cens(y=wave, sr=sr,n_chroma=40)
    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]   
        
    if (max_len > melspectrogram.shape[1]):
        pad_width = max_len - melspectrogram.shape[1]
        melspectrogram = np.pad(melspectrogram, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        melspectrogram = melspectrogram[:, :max_len]   
        
    if (max_len > chroma_stft.shape[1]):
        pad_width = max_len - chroma_stft.shape[1]
        chroma_stft = np.pad(chroma_stft, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        chroma_stft = chroma_stft[:, :max_len]   

        
    if (max_len > chroma_cq.shape[1]):
        pad_width = max_len - chroma_cq.shape[1]
        chroma_cq = np.pad(chroma_cq, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        chroma_cq = chroma_cq[:, :max_len]   
        
    if (max_len > chroma_cens.shape[1]):
        pad_width = max_len - chroma_cens.shape[1]
        chroma_cens = np.pad(chroma_cens, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        chroma_cens = chroma_cens[:, :max_len]   
    # Else cutoff the remaining parts
    features=np.stack((mfcc,melspectrogram,chroma_stft,chroma_cq,chroma_cens),axis=2)
    
    return features

In [5]:
mfcc = wav2mfcc("/media/tintin/3bde46c1-cca7-4db6-b2ca-e71397572866/tintin/speech-urban-data/speech/data/bed/00176480_nohash_0.wav")
print(mfcc.shape)

/home/tintin/miniconda3/envs/tf36/lib/python3.7/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


(40, 11, 5)


In [6]:
# args: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
# make sure the background noise folder is not insde the DATA_PATH. because that is used for data augmentation and not recognition
def get_labels(path=COMBINED_DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


In [ ]:
# write the save data code here....with test train and val split
# make sure the background_noise folder is not in the path
def save_data_speech(path = SPEECH_DATA_PATH, testFile = testFile, valFile = valFile, max_len = 11,savepath = SPEECH_NPY_PATH):

    test_file = open(testFile, "r")
    testFilesList = test_file.read().split('\n')

    val_file = open(valFile, "r")
    valFilesList = val_file.read().split('\n')

    #print(testFilesList)
    #print(valFilesList)
    labels,_,_ = get_labels(path)
    print(labels)
    for label in labels:
        mfcc_train = []
        mfcc_test = []
        mfcc_val = []
        # saving a tuple of wavfile path and label/name format to compare in the test and val list
        wavfiles = [(path + label + '/' + wavfile, label + '/' + wavfile)
                    for wavfile in os.listdir(path + '/' + label)]
        
        #print(wavfiles)
        
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            #print(wavfile[0])
            #print(wavfile[1])
            mfcc = wav2mfcc(wavfile[0], max_len=max_len)
            #print(mfcc.shape)
            if wavfile[1] in testFilesList:
                mfcc_test.append(mfcc)
            elif wavfile[1] in valFilesList:
                mfcc_val.append(mfcc)
            else:
                mfcc_train.append(mfcc)
                
        np.save(savepath + label + '_test.npy', mfcc_test)
        np.save(savepath + label + '_val.npy', mfcc_val)
        np.save(savepath + label + '_train.npy', mfcc_train)

In [ ]:
# just saving the urban data as the npy file.
# will split the data into test train and val after loading the data nd the labels by using a test_train split function
def save_urbanNoise_data(path = URBAN_NOISE_DATA_PATH, max_len = 11, savePath = URBAN_NOISE_NPY_PATH):
    labels,_,_ = get_labels(path)
    for label in labels:
        mfccs = []
        mfcc_train = []
        mfcc_test = []
        mfcc_val = []
        print(label)
        
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        
        for wavfile in tqdm(wavfiles, "saving vectors of label - '{}'".format(label)):
            try:
                mfcc = wav2mfcc(wavfile, max_len = max_len)
                mfccs.append(mfcc)
            except:
                print(wavfile)
        
        np.save(savePath + label + '.npy', mfccs)

In [7]:
# save data
# Second dimension of the feature is dim2
feature_dim_2 = 40      #max_len

In [ ]:


# speech data
save_data_speech(max_len = feature_dim_2)


In [ ]:
#urban Noise data
save_urbanNoise_data(max_len = feature_dim_2)

### Loading saved data

In [8]:
# get the urbanNoiseData along with their labels then split them into train,val and split and save as npy files
# we will get all the data from all the classes append them into an array and then use a test train split function
def getUrbanNoiseDataSplit(path = URBAN_NOISE_DATA_PATH, savePath = URBAN_NOISE_NPY_PATH, split_ratio=0.8,random_state = 42):
    labels, indices, _ = get_labels(path)
    # get the array for each label
    X = np.load(savePath + labels[0] + '.npy')
    Y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    # filling with i+1 in each loop so basically starting the class values from 0 to 9.
    for i, label in enumerate(labels[1:]):
        x = np.load(savePath + label + '.npy')
        X = np.vstack((X, x))
        Y = np.append(Y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(Y)
    
    # not shuffling the data for now. Will shuffle them while training
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp,Y_temp, test_size = (1-split_ratio), random_state = random_state,shuffle = True)
    
    return labels,X_train,X_val,X_test,Y_train,Y_val,Y_test

In [9]:
# for speech we already split the data into train test and val and then saved the data. 
# so now we just need to import the data and then append all the labels
def getSpeechDataSplit(path = SPEECH_DATA_PATH, savePath = SPEECH_NPY_PATH):
    
    labels, indices, _ = get_labels(path)
    # get the array for each label
    X_train = np.load(savePath + labels[0] + '_train.npy')
    Y_train = np.zeros(X_train.shape[0])
    X_val = np.load(savePath + labels[0] + '_val.npy')
    Y_val = np.zeros(X_val.shape[0])
    X_test = np.load(savePath + labels[0] + '_test.npy')
    Y_test = np.zeros(X_test.shape[0])
    
    for i,label in enumerate(labels[1:]):
        x_train = np.load(savePath + label + '_train.npy')
        x_val = np.load(savePath + label + '_val.npy')
        x_test = np.load(savePath + label + '_test.npy')
        X_train = np.vstack((X_train, x_train))
        X_val = np.vstack((X_val, x_val))
        X_test = np.vstack((X_test, x_test))
        
        Y_train = np.append(Y_train, np.full(x_train.shape[0],fill_value= (i+1)))
        Y_val = np.append(Y_val, np.full(x_val.shape[0],fill_value= (i+1)))
        Y_test = np.append(Y_test, np.full(x_test.shape[0],fill_value= (i+1)))
    
        
    return labels,X_train,X_val,X_test,Y_train,Y_val,Y_test

In [10]:
# # Loading train set and test set of urban Noise
labels_un, X_train_un,X_val_un, X_test_un, Y_train_un,Y_val_un, Y_test_un = getUrbanNoiseDataSplit()
print(labels_un)

['air_conditioner', 'dog_bark', 'street_music', 'car_horn', 'drilling', 'children_playing', 'siren', 'engine_idling', 'jackhammer', 'gun_shot']


In [11]:
# # Loading train set and test set of speech
labels_sp, X_train_sp,X_val_sp, X_test_sp, Y_train_sp,Y_val_sp, Y_test_sp = getSpeechDataSplit()
print(labels_sp)

['down', 'learn', 'right', 'nine', 'eight', 'dog', 'bird', 'house', 'marvin', 'zero', 'sheila', 'bed', 'follow', 'off', 'happy', 'backward', 'on', 'cat', 'left', 'five', 'visual', 'one', 'no', 'two', 'yes', 'forward', 'tree', 'three', 'go', 'seven', 'six', 'wow', 'stop', 'four', 'up']


In [12]:
# print shapes of all arrays
X_train_un.shape,X_val_un.shape, X_test_un.shape, Y_train_un.shape,Y_val_un.shape, Y_test_un.shape

((5579, 40, 40, 5),
 (1395, 40, 40, 5),
 (1744, 40, 40, 5),
 (5579,),
 (1395,),
 (1744,))

In [13]:
X_train_sp.shape,X_val_sp.shape, X_test_sp.shape, Y_train_sp.shape,Y_val_sp.shape, Y_test_sp.shape

((84843, 40, 40, 5),
 (9981, 40, 40, 5),
 (11005, 40, 40, 5),
 (84843,),
 (9981,),
 (11005,))

In [14]:
# if we want to combine both the samples we just need to add the Y sample with the numeber of labels in the previous dataset

# combining samples
labels = labels_sp + labels_un
print(len(labels))



45


### Merging data

In [15]:
# we add 35 to the urban Noise Y data to make them into the new labels


Y_train_un_new = Y_train_un + 35
Y_test_un_new = Y_test_un + 35
Y_val_un_new = Y_val_un + 35



In [16]:
Y_train = np.append(Y_train_sp, Y_train_un_new)
Y_test = np.append(Y_test_sp, Y_test_un_new)
Y_val = np.append(Y_val_sp, Y_val_un_new)

print(Y_train.shape)
print(Y_test.shape)
print(Y_val.shape)

(90422,)
(12749,)
(11376,)


In [17]:
print(Y_test)
print(Y_train)
print(Y_val)

[ 0.  0.  0. ... 39. 37. 38.]
[ 0.  0.  0. ... 37. 39. 40.]
[ 0.  0.  0. ... 38. 44. 36.]


In [18]:
X_train = np.vstack((X_train_sp, X_train_un))
X_test = np.vstack((X_test_sp, X_test_un))
X_val = np.vstack((X_val_sp, X_val_un))


print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(90422, 40, 40, 5)
(12749, 40, 40, 5)
(11376, 40, 40, 5)


In [ ]:
# we have combined data set and seperate datasets
# X_train_un, X_test_un, X_val_un, Y_train_un, Y_test_un, Y_val_un ---urban Noise data - 10 clasees - (0-9)
# X_train_sp, X_test_sp, X_val_sp, Y_train_sp, Y_test_sp, Y_val_sp --- speech data - 35 classes - (0-34)
# X_train, X_test, X_val, Y_train, Y_test, Y_val --- combined dtat - 45 classes (0-44)

In [19]:
# # Feature dimension
feature_dim_1 = 40
channel = 5
epochs = 200
batch_size = 100
verbose = 1
num_classes = 45    #keeps changing


In [20]:
# Reshaping to perform 2D convolution
# all data

X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_val = X_val.reshape(X_val.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

Y_train_hot = to_categorical(Y_train)
Y_val_hot = to_categorical(Y_val)
Y_test_hot = to_categorical(Y_test)

In [21]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

print(Y_train_hot.shape)
print(Y_test_hot.shape)
print(Y_val_hot.shape)

(90422, 40, 40, 5)
(12749, 40, 40, 5)
(11376, 40, 40, 5)
(90422, 45)
(12749, 45)
(11376, 45)


### Model definition

In [22]:
def get_model(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

### Training

In [28]:


model = get_model(45)

# checkpoint
filepath=COMBINED_MODEL_SAVE_PATH + "all_data-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, Y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, 
                    validation_data=(X_test, Y_test_hot), callbacks=callbacks_list, )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 90422 samples, validate on 12749 samples
Epoch 1/200
  500/90422 [..............................] - ETA: 2:08:42 - loss: 3.9350 - acc: 0.0280

/home/tintin/miniconda3/envs/tf36/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.298519). Check your callbacks.
  % delta_t_median)


90422/90422 [==============================] - 85s 940us/step - loss: 2.7526 - acc: 0.2221 - val_loss: 1.6007 - val_acc: 0.4985

Epoch 00001: val_acc improved from -inf to 0.49847, saving model to /media/tintin/3bde46c1-cca7-4db6-b2ca-e71397572866/tintin/speech-urban-data/combined/4040binsextra/all_data-01-0.50.hdf5
Epoch 2/200
90422/90422 [==============================] - 40s 439us/step - loss: 1.5773 - acc: 0.5317 - val_loss: 1.1057 - val_acc: 0.6631

Epoch 00002: val_acc improved from 0.49847 to 0.66311, saving model to /media/tintin/3bde46c1-cca7-4db6-b2ca-e71397572866/tintin/speech-urban-data/combined/4040binsextra/all_data-02-0.66.hdf5
Epoch 3/200
90422/90422 [==============================] - 40s 439us/step - loss: 1.2526 - acc: 0.6365 - val_loss: 0.9169 - val_acc: 0.7296

Epoch 00003: val_acc improved from 0.66311 to 0.72963, saving model to /media/tintin/3bde46c1-cca7-4db6-b2ca-e71397572866/tintin/speech-urban-data/combined/4040binsextra/all_data-03-0.73.hdf5
Epoch 4/200
9042

90422/90422 [==============================] - 39s 433us/step - loss: 0.5306 - acc: 0.8547 - val_loss: 0.6615 - val_acc: 0.8376

Epoch 00070: val_acc did not improve from 0.84242
Epoch 71/200
90422/90422 [==============================] - 40s 437us/step - loss: 0.5362 - acc: 0.8544 - val_loss: 0.7005 - val_acc: 0.8320

Epoch 00071: val_acc did not improve from 0.84242
Epoch 72/200
90422/90422 [==============================] - 40s 437us/step - loss: 0.5348 - acc: 0.8539 - val_loss: 0.7283 - val_acc: 0.8305

Epoch 00072: val_acc did not improve from 0.84242
Epoch 73/200
90422/90422 [==============================] - 40s 438us/step - loss: 0.5317 - acc: 0.8565 - val_loss: 0.7142 - val_acc: 0.8241

Epoch 00073: val_acc did not improve from 0.84242
Epoch 74/200
90422/90422 [==============================] - 40s 437us/step - loss: 0.5253 - acc: 0.8575 - val_loss: 0.6755 - val_acc: 0.8380

Epoch 00074: val_acc did not improve from 0.84242
Epoch 75/200
90422/90422 [===========================

90422/90422 [==============================] - 39s 435us/step - loss: 0.5346 - acc: 0.8570 - val_loss: 0.7276 - val_acc: 0.8321

Epoch 00112: val_acc did not improve from 0.84548
Epoch 113/200
90422/90422 [==============================] - 39s 434us/step - loss: 0.5360 - acc: 0.8580 - val_loss: 0.6778 - val_acc: 0.8292

Epoch 00113: val_acc did not improve from 0.84548
Epoch 114/200
90422/90422 [==============================] - 39s 433us/step - loss: 0.5282 - acc: 0.8577 - val_loss: 0.7906 - val_acc: 0.8152

Epoch 00114: val_acc did not improve from 0.84548
Epoch 115/200
90422/90422 [==============================] - 39s 434us/step - loss: 0.5437 - acc: 0.8569 - val_loss: 0.6700 - val_acc: 0.8332

Epoch 00115: val_acc did not improve from 0.84548
Epoch 116/200
90422/90422 [==============================] - 40s 439us/step - loss: 0.5424 - acc: 0.8567 - val_loss: 0.6896 - val_acc: 0.8329

Epoch 00116: val_acc did not improve from 0.84548
Epoch 117/200
90422/90422 [======================

KeyboardInterrupt: 

In [23]:
# loading the model with the best val accuracy
model = get_model()
weights_path = "./models/model2b.hdf5"
model.load_weights(weights_path)
model.summary()

TypeError: get_model() missing 1 required positional argument: 'num_classes'

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(COMBINED_MODEL_SAVE_PATH + 'model_accuracy.png')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(COMBINED_MODEL_SAVE_PATH + 'model_loss.png')
plt.show()


In [ ]:
#train and test loss and scores respectively
train_loss_score=model.evaluate(X_train,Y_train_hot)
val_loss_score=model.evaluate(X_val,Y_val_hot)
test_loss_score=model.evaluate(X_test,Y_test_hot)
print(train_loss_score)
print(val_loss_score)
print(test_loss_score)

In [ ]:
# Predicts one sample
labels = ['down', 'learn', 'right', 'nine', 'eight', 'dog', 'bird', 'house',
 'marvin', 'zero', 'sheila', 'bed', 'follow', 'off', 'happy', 'backward', 'on',
  'cat', 'left', 'five', 'visual', 'one', 'no', 'two', 'yes', 'forward', 'tree',
   'three', 'go', 'seven', 'six', 'wow', 'stop', 'four', 'up','air_conditioner',
   'dog_bark', 'street_music', 'car_horn', 'drilling', 'children_playing',
    'siren', 'engine_idling', 'jackhammer', 'gun_shot']

def predict(filepath, model,labels):
    mfcc = wav2mfcc(testFile, max_len=40)
    mfcc_reshaped = mfcc.reshape(1,feature_dim_1,feature_dim_2,channel)
    odds= model.predict(mfcc_reshaped)
    odds_max = np.argmax(odds)
    label = labels[odds_max]
    return label

In [ ]:
testFile = SPEECH_DATA_PATH + 'happy/27c30960_nohash_0.wav'
print(predict(testFile,model,labels))